In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_batch(batchsize=32, validation=False):
    get_case = get_validation_case if validation else get_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        song, tempo, compatible, clip = get_case()
        c = clip_to_tf_input(resample_clip(clip))
        x = np.expand_dims(c, axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(compatible)).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch(batchsize, validation=True))

In [3]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [4]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [5]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, activation='relu', input_shape=input_shape))
model.add(keras.layers.Conv2D(32, 3, activation='relu'))
model.add(keras.layers.Conv2D(16, 5, activation='relu'))
model.add(keras.layers.Conv2D(8, 7, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        12816     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 8)           6280      
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 1032      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total para

In [8]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=8,
                    epochs=7, 
                    steps_per_epoch=16)

Epoch 1/7
16/16 [==============================] - 166s 10s/step - loss: 0.8313 - acc: 0.6309 - val_loss: 0.6280 - val_acc: 0.7695
Epoch 2/7
16/16 [==============================] - 122s 8s/step - loss: 0.6095 - acc: 0.7539 - val_loss: 0.6834 - val_acc: 0.7422
Epoch 3/7
16/16 [==============================] - 121s 8s/step - loss: 0.6064 - acc: 0.7949 - val_loss: 0.6786 - val_acc: 0.7422
Epoch 4/7
16/16 [==============================] - 122s 8s/step - loss: 0.6136 - acc: 0.7832 - val_loss: 0.6275 - val_acc: 0.7656
Epoch 5/7
16/16 [==============================] - 112s 7s/step - loss: 0.5724 - acc: 0.8047 - val_loss: 0.6446 - val_acc: 0.7227
Epoch 6/7
16/16 [==============================] - 124s 8s/step - loss: 0.6006 - acc: 0.8145 - val_loss: 0.6993 - val_acc: 0.7422
Epoch 7/7
16/16 [==============================] - 119s 7s/step - loss: 0.5852 - acc: 0.7852 - val_loss: 0.6204 - val_acc: 0.7656


In [16]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

IfIHadAHammer 133
Predicted:  0.434026    Actual:  False 

ItsRainingMen 91
Predicted:  0.2488131    Actual:  False 

EternalFlame 225
Predicted:  0.44283226    Actual:  False 

IllBeThereForYou 64
Predicted:  0.10472007    Actual:  False 

ImLookingThruYou 71
Predicted:  0.4656722    Actual:  False 

IfIHadAHammer 133
Predicted:  0.43624005    Actual:  False 

Kalamazoo 61
Predicted:  0.37418914    Actual:  False 

ImLookingThruYou 86
Predicted:  0.46337795    Actual:  True 

WalkThisWay1 63
Predicted:  0.22730537    Actual:  False 

Kalamazoo 69
Predicted:  0.3396961    Actual:  True 

